# 02 Gridding locations

Analysis reads in locations identified for publications, de-duplicates locations by publication, identifies shapefiles from natural earth for locations, and then bin the shapefiles to the grid. If no shapefile is found, then the location is mapped to the grid cell which covers the centroid of the location. 

*Note to Vicky*
Also, it would be great to store the area of each polygon before binning and store it in a separate data frame so we can examine the spatial scale of different publications. What do you think?

In [1]:
# First we load our location data and drop any duplicated places within documents
import pandas as pd
import shapely.vectorized
from global_land_mask import globe
import os
import pandas as pd
import warnings
import time
import numpy as np
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import geopandas
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colormaps

In [2]:
## Setup parameters
gridRes = 2.5 # the desired resolution of the grid to map to

In [3]:
## Load in geoparsed location data

# Set the directory path where your CSV files are located
directory_path = '/home/dveytia/test-mordecai/outputs/geoparsed-text'
dfs = [] # initialise empty list

# Loop through all files in the directory and load
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory_path, filename)
        # Read each CSV file into a DataFrame and append it to the list
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenate all DataFrames in the list into one
places = pd.concat(dfs, ignore_index=True)
places = places.drop_duplicates(["id","geonameid"]) 
places = places.rename(columns={"id": "analysis_id"})
print(places.shape)
places.head(2)

(44127, 15)


,analysis_id,duplicate_id,title,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code
0,369746,1997.4164,Study on multi-function ocean thermal energy c...,Fiji,"[{'start': 783, 'end': 787}]",FJI,0.946707,NaN,-18.0000,178.000,FJI,2205218,Republic of Fiji,A,PCLI
1,369760,1997.4175,Role of sewage phosphorus in coastal water ene...,Tuticorin,"[{'start': 133, 'end': 142}]",IND,0.952811,Tamil Nadu,8.8375,77.963,IND,7627069,Thoothukkudi,A,ADM2


In [6]:
## If you want to by-pass the next sections, load outputs directly:
grid_df = pd.read_csv(f'/home/dveytia/test-mordecai/outputs/grid_df_res{gridRes}.csv', low_memory=False) # the grid
shp_df = pd.read_csv(f'/home/dveytia/test-mordecai/outputs/shp_df_natural-earth-shapes.csv', low_memory=False) # natural earth shapefiles
shp_grid_df = pd.read_csv(f'/home/dveytia/test-mordecai/outputs/shp_grid_df.csv', low_memory=False) # shapefiles matched to grid
shp_df_matches = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_shp_df_matches.csv", low_memory=False) # text match to shapefiles

# Set up grid to map to

In this section, set up a grid to map to based on the desired resolution, calculate the area of each grid cell, and also determine if a grid cell is on land or not. 

In [9]:
## Generate grid to map to
def generate_grid_df(degrees):
    '''
    Generate a dataframe with a grid of of cells degrees x degrees
    '''
    LON = np.linspace(-180+degrees*0.5,180-degrees*0.5,int(360/degrees))
    LAT = np.linspace(-90+degrees*0.5,90-degrees*0.5,int(180/degrees))
    lon_df, lat_df = np.meshgrid(LON,LAT)

    return pd.DataFrame({"LAT": lat_df.ravel(), "LON": lon_df.ravel()})
    
grid_df = generate_grid_df(gridRes)

grid_df['grid_df_id'] = range(len(grid_df))

print(grid_df.head(3))
print(grid_df.shape) # (10368, 3)

     LAT     LON  grid_df_id
0 -88.75 -178.75           0
1 -88.75 -176.25           1
2 -88.75 -173.75           2
(10368, 3)


In [10]:
## Calculate area of each grid cell

# Define function to calculate the area of a gridcell given the center lat and lon and the size in degrees
import math
def area_cell(lat, lon, degrees): 
    # calculate the area of a gridcell given the center lat and lon and the size in degrees
    if lon <0:
        lon+=360
    R = 6371
    f0 = math.radians(lat-degrees*0.5)
    f1 = math.radians(lat+degrees*0.5)
    l0 = math.radians(lon-degrees*0.5)
    l1 = math.radians(lon+degrees*0.5)

    return (math.sin(f1)-math.sin(f0)) * (l1 - l0) * R**2

# Calculate area of each grid cell
grid_df['area_km'] = grid_df.apply(lambda x: area_cell(x['LAT'], x['LON'], gridRes), axis=1)

In [11]:
## Determine which grid cells are on land

# Create a list to determine if a cell is on land
land_masks = []
n = 5
land_array = np.empty((grid_df.shape[0], n*n))
i = 0
for x in np.linspace(0,2.5,n):
    for y in np.linspace(0,2.5,n):
        land_array[:,i] = globe.is_land(grid_df.LAT+y-1.25, grid_df.LON+x-1.25)
        i+=1
        
land_mask = land_array.sum(axis=1)>0

# Add a column indicating if the grid cell is on land
grid_df['is_land'] = land_mask
grid_df.loc[grid_df['LAT']<-60,'is_land'] = False

# Print how many cells are on land -- 3311
grid_df.is_land.sum() 
grid_df.head()

,LAT,LON,grid_df_id,area_km,is_land
0,-88.75,-178.75,0,1685.654015,False
1,-88.75,-176.25,1,1685.654015,False
2,-88.75,-173.75,2,1685.654015,False
3,-88.75,-171.25,3,1685.654015,False
4,-88.75,-168.75,4,1685.654015,False


In [12]:
## Export grid to reference file
grid_df.to_csv(f'/home/dveytia/test-mordecai/outputs/grid_df_res{gridRes}.csv',index=False)

# Read in shapefiles from Natural Earth

Create a lookup table for each natural earth shape file and grid cell. Read in shapefiles from Natural Earth, and based on the grid, and determine which grid cells correspond to which shape files.

In [13]:
## Read in shapefiles from natural earth
# First we define a list of the shapefile definitions we want
shpfiles = [
    dict(resolution='50m', category='cultural', name='admin_0_countries'),
    dict(resolution='10m', category='cultural', name='admin_1_states_provinces'),
    dict(resolution='10m', category='physical', name='geography_regions_polys'),
    dict(resolution='10m', category='physical', name='geography_marine_polys')
    #"data/gadm36_1.shp"
]

# We'll start an empty dataframe to store our shapefile-grid matches
shp_grid_df = pd.DataFrame()

# Now we download the shapefiles and combine into one large geopandas dataframe
shp_df  = None
for shpfilename in shpfiles:
    print(f"reading shapefile {shpfilename}")
    if shp_df is None:
        shp_df = geopandas.read_file(shpreader.natural_earth(**shpfilename))
    else:
        shp_df = pd.concat([shp_df, geopandas.read_file(shpreader.natural_earth(**shpfilename))])
        #shp_df = shp_df.merge(geopandas.read_file(shpreader.natural_earth(**shpfilename)),how="outer")
    time.sleep(10) # Wait a bit before downloading the next shapefile so we do not make too many requests too quickly

reading shapefile {'resolution': '50m', 'category': 'cultural', 'name': 'admin_0_countries'}
reading shapefile {'resolution': '10m', 'category': 'cultural', 'name': 'admin_1_states_provinces'}
reading shapefile {'resolution': '10m', 'category': 'physical', 'name': 'geography_regions_polys'}
reading shapefile {'resolution': '10m', 'category': 'physical', 'name': 'geography_marine_polys'}


In [16]:
shp_df['shpfile_id'] = range(len(shp_df))
print(shp_df.shape)
print(shp_df.head(2))

(6191, 264)
        featurecla  scalerank  LABELRANK SOVEREIGNT SOV_A3  ADM0_DIF  LEVEL  \
0  Admin-0 country        1.0        3.0   Zimbabwe    ZWE       0.0    2.0   
1  Admin-0 country        1.0        3.0     Zambia    ZMB       0.0    2.0   

                TYPE TLC     ADMIN  ... name_zht  FEATURECLA NAMEALT REGION  \
0  Sovereign country   1  Zimbabwe  ...      NaN         NaN     NaN    NaN   
1  Sovereign country   1    Zambia  ...      NaN         NaN     NaN    NaN   

   SCALERANK LABEL namealt  changed label shpfile_id  
0        NaN   NaN     NaN      NaN   NaN          0  
1        NaN   NaN     NaN      NaN   NaN          1  

[2 rows x 264 columns]


/opt/miniconda3/envs/mordecai_env_3.8/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


In [45]:
## Calculate area of each polygon in km2
from pyproj import Geod
from shapely import wkt

def area_poly(poly): 
    # specify a named ellipsoid
    geod = Geod(ellps="WGS84")
    # calculate area
    return abs(geod.geometry_area_perimeter(poly)[0])/1000000


shp_df['area_km2'] = shp_df.apply(lambda x: area_poly(x['geometry']), axis=1)

## NB if I want to check, here is a quick way to plot the polygon
# import matplotlib.pyplot as plt
# poly = shp_df['geometry'].iloc[0]
# plt.plot(*poly.exterior.xy)

shp_df['area_km2'].describe()

count    6.191000e+03
mean     1.738410e+05
std      2.007239e+06
min      1.007477e-06
25%      6.902486e+02
50%      5.102613e+03
75%      2.997981e+04
max      8.010105e+07
Name: area_km2, dtype: float64

In [46]:
## Save natural earth shapefiles
shp_df.to_csv(f'/home/dveytia/test-mordecai/outputs/shp_df_natural-earth-shapes.csv',index=False)

In [47]:
## Match shapefiles to gridcell indices 

# We are going to store our shapefile-gridcell index matches here
shp_grid = []

# This is the grid we will work with
yv, xv = np.meshgrid(grid_df.LAT.unique(), grid_df.LON.unique())
for i, place in shp_df.iterrows(): # Now we go through all the shapes
    # show which gridcell centers are contained inside the shape
    # ignore the warning caused by shapely using an old version of numpy
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        inplace = shapely.vectorized.contains(place.geometry, xv, yv)
    idx = np.argwhere(inplace)
    # Get the number of cells contained in the shape
    number_cells = idx.size/2
    if number_cells == 0:
        # If we have no cell centers in the shape, get the shape center and the cell which contains it
        c = place.geometry.centroid
        lon = c.x//gridRes*gridRes+gridRes*0.5
        lat = c.y//gridRes*gridRes+gridRes*0.5
        da_df = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)]
        shp_grid.append({"shpfile_id": i, "grid_df_id": da_df.index[0]})
    else:
        for point in idx:
            lon = grid_df.LON.unique()[point[0]]
            lat = grid_df.LAT.unique()[point[1]]
            da_df = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)]
            shp_grid.append({"shpfile_id": i, "grid_df_id": da_df.index[0]}) 

shp_grid_df = pd.DataFrame.from_dict(shp_grid)
    

shp_grid_df.head(2)

,shpfile_id,grid_df_id
0,0,4114
1,0,3971


In [48]:
shp_grid_df.to_csv(f'/home/dveytia/test-mordecai/outputs/shp_grid_df.csv',index=False)

# Document to grid matching

Look for place matches between mordecai geoparsing results and the shape files, and then map these results to grid cells. Then sum the number of documents found in each grid cell

In [7]:
feature_mapping = {
    "ADM1": ["Admin-1 scale rank", "Admin-1 aggregation", "Admin-1 minor island"],
    "PCLI": ["Admin-0 country"],
    'MTS': ['Range/mtn'],
    'PLAT': ['Plateau'],
    'PLN': ['Plain'],
    'DSRT': ['Desert'],
    'OCN': ['ocean'],
    'SEA': ['sea', 'bay'],
    'GULF': ['gulf', 'bay'],
    'BAY': ['gulf', 'bay'],
    'CHN': ['channel'],
    'BSNU': ['basin']
}

In [8]:
# We add all the other codes we don't cover with blank shapefile classes
for fcode in places.feature_code.unique():
    if fcode not in feature_mapping:
        feature_mapping[fcode] = []

In [9]:
# To help match, we will rename some shapes so that they are the same as the name in our database
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Altay", case=False)),"name"] = "Altay"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Appalach", case=False)),"name"] = "Appalachian Mountains"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("cant", case=False)),"name"] = "Cordillera Cantábrica"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Dabie", case=False)),"name"] = "Dabie Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("EASTERN GHATS", case=False)),"name"] = "Eastern Ghāts"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("WESTERN GHATS", case=False)),"name"] = "Western Ghāts"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("kunlun", case=False)),"name"] = "Kalakunlun Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("LEN MOUNTAIN", case=False)),"name"] = "Kölen"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Taihang Mts.", case=False)),"name"] = "Taihang Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Tatra Mts.", case=False)),"name"] = "Tatry"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("TIAN SHAN", case=False)),"name"] = "Tien Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("andes", case=False)),"name"] = "Andes Mountains"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("HINDU KUSH", case=False)),"name"] = "Hindū Kush"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Marrah Mts", case=False)),"name"] = "Jabal Marrah"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Lebanon", case=False)),"name"] = "Mount Lebanon"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("KARAKORAM RA", case=False)),"name"] = "Karakorum Shan"

shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Negev", case=False)), "name"] = "Negev"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Atacama", case=False)), "name"] = "Atacama Desert"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("CHIHUAHUAN DESERT", case=False)), "name"] = "Chihuahua Desert"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Lut desert", case=False)), "name"] = "God-e Lut"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("TAKLIMAKAN DESERT", case=False)), "name"] = "Takla Makan Desert"

shp_df.loc[
    (shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("cumberland", case=False)),["name","featurecla"]
] = ["Cumberland Plateau", "Plain"]
shp_df.loc[
    (shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("colorado", case=False)),["name","featurecla"]
] = ["San Francisco Plateau", "Plain"]
shp_df.loc[(shp_df["featurecla"]=="Plain") & (shp_df["name"].str.contains("gange", case=False)),"name"] = "Gangetic Plain"
shp_df.loc[(shp_df["featurecla"]=="Plain") & (shp_df["name"].str.contains("north china", case=False)),"name"] = "Huanghuai Pingyuan"

shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("mongol", case=False)), "name"] = "Nei Mongol Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("deccan", case=False)), "name"] = "Deccan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("chota", case=False)), "name"] = "Chota Nāgpur Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("loess", case=False)), "name"] = "Huangtu Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("khorat", case=False)), "name"] = "Khorat Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("tibet", case=False)), "name"] = "Qing Zang Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("polar", case=False)), "name"] = "South Polar Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("YUNGUI", case=False)), "name"] = "Yungui Gaoyuan"

places["place_name"] = places["place_name"].str.lower().str.replace("mts.","mountains") 
shp_df["name"] = shp_df["name"].str.lower().str.replace("mts.","mountains") 

/tmp/ipykernel_3362076/2263670162.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
  places["place_name"] = places["place_name"].str.lower().str.replace("mts.","mountains")
/tmp/ipykernel_3362076/2263670162.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
  shp_df["name"] = shp_df["name"].str.lower().str.replace("mts.","mountains")


In [ ]:
## NB this takes several min to run
## For each document that has a place, search for a match from the natural earth shapefiles and it's corresponding grid cell indexes
## Reminder the object 'places' is a data frame that contains all the document information with places identified

# We'll start off with an empty dataframe
shp_df_matches = pd.DataFrame()

# Now we want to look through the feature type dictionary we had before
for place_key, shpfile_keys in feature_mapping.items():
    
    #print(place_key)
    
    # We can get all the places and all the shapes
    feature_places = places.loc[places["feature_code"]==place_key]
    feature_shapes = shp_df.loc[shp_df["featurecla"].isin(shpfile_keys)]
    
    # And loop through the places
    for place, group in feature_places.groupby("place_name"):
        # If we don't have a shapefile feature type we just take the grid cell containing the point
        if not shpfile_keys:
            shp_id = None # we set shp_id to None, round the coordinates, and take the gridcell which has these coordinates
            lon = group.lon.values[0]//gridRes*gridRes+gridRes*0.5
            lat = group.lat.values[0]//gridRes*gridRes+gridRes*0.5
            grid_df_ids = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)].index

        else:
            # Otherwise, we are going to try to find the places which match
            place_shapes = pd.DataFrame()
            # First we will try by geoname ids, if we have these
            if feature_shapes["gn_id"].sum() > 0:
                place_shapes = feature_shapes.loc[
                    (feature_shapes["gn_id"]==group.geonameid.values[0])
                ]
            # Then we try by country code, if we are dealing with a country
            if place_shapes.shape[0]==0 and place_key=="PCLI":
                place_shapes = feature_shapes.loc[
                    feature_shapes["ADM0_A3"] == group.country_predicted.values[0]
                ]
            # Then we try by name
            if place_shapes.shape[0]==0:       
                place_shapes = feature_shapes.loc[
                    (feature_shapes.name == place)
                ]    

            if place_shapes.shape[0]==0:
                continue # These are the places we cannot match, we need to develop other strategies for these, e.g. using other shapefile libraries
    #            if key=="ADM1": # can also try using gadm data, which we need to download separately
    #                 place_shapes = adm1shps_alt[
    #                     (adm1shps_alt["NAME_1"]==group.place_name.values[0]) |
    #                     (adm1shps_alt["VARNAME_1"].str.contains(group.place_name.values[0]))
    #                 ]
            else:
                shp_id = place_shapes.index[0]
                grid_df_ids = shp_grid_df.loc[
                    (shp_grid_df["shpfile_id"]==shp_id),
                    "grid_df_id"
                ]
        # For each document with this place, we add a row for each grid cell index matching the place
        for did in group.analysis_id.unique():
            shp_df_matches = pd.concat([
                shp_df_matches,
                pd.DataFrame.from_dict({
                    "grid_df_id": grid_df_ids,
                    "analysis_id": [did] * len(grid_df_ids),
                    "shp_id": [shp_id] * len(grid_df_ids),
                    "place": place
                })
            ])
            
            
print(shp_df_matches.shape)
shp_df_matches.head(2)

(944534, 4)


,grid_df_id,analysis_id,shp_id,place
2488,6095,284452.0,232.0,antigua and barbuda
3895,7568,284452.0,232.0,antigua and barbuda


In [21]:
shp_df_matches = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_shp_df_matches.csv", low_memory = False)

## For each unique document and place, calculate the 1/the number of grid cells occupied by that place to get the grid cell weight
shp_df_weight = shp_df_matches.groupby(['analysis_id','shp_id']).apply(lambda x: 1/len(x)).reset_index() # calculate 1/number of grid cells per place
shp_df_weight = shp_df_weight.set_axis(['analysis_id','shp_id','cell_weight'], axis=1) # rename columns
shp_df_matches = shp_df_matches.merge(shp_df_weight, how='left') # Merge back into the corresponding grid cells


print(shp_df_matches.shape)
shp_df_matches.head(2)

(944534, 5)


,grid_df_id,analysis_id,shp_id,place,cell_weight
0,6095,284452.0,232.0,antigua and barbuda,0.25
1,7568,284452.0,232.0,antigua and barbuda,0.25


In [22]:
## Write to csv
shp_df_matches.to_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_shp_df_matches.csv",index=False)

In [12]:
grid_df.head()

,LAT,LON,grid_df_id,area_km,is_land
0,-88.75,-178.75,0,1685.654015,False
1,-88.75,-176.25,1,1685.654015,False
2,-88.75,-173.75,2,1685.654015,False
3,-88.75,-171.25,3,1685.654015,False
4,-88.75,-168.75,4,1685.654015,False


# Read in document to shpfile matches and sum number of matches per cell

In [29]:
## Read in 
shp_df_matches = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_shp_df_matches.csv", low_memory = False)

## For each grid cell, sum the number of documents 
shp_df_sum = shp_df_matches.groupby(['grid_df_id']).apply(lambda x: len(x)).reset_index()
shp_df_sum = shp_df_sum.set_axis(['grid_df_id','n_articles'], axis=1)

## For each grid cell, sum the weighted number of documents to get the weighted sum
shp_df_weightedSum = shp_df_matches.groupby(['grid_df_id'])['cell_weight'].sum().reset_index()
shp_df_weightedSum = shp_df_weightedSum.set_axis(['grid_df_id','n_articles_weighted'], axis=1)


## Merge all together with the grid cell locations 
# Format the grid so the id column is identiical to allow for a merge
grid_df2 = grid_df
grid_df2['grid_df_id'] = grid_df.index

# Merge
shp_df_sum_grid = grid_df2.merge(shp_df_sum, how = "left")
shp_df_sum_grid = shp_df_sum_grid.merge(shp_df_weightedSum, how = "left")

# Summarise
shp_df_sum_grid['n_articles'] = shp_df_sum_grid['n_articles'].fillna(0) # set NAs to 0
shp_df_sum_grid['n_articles_weighted'] = shp_df_sum_grid['n_articles_weighted'].fillna(0) # set NAs to 0
print(shp_df_sum_grid.shape)
print(shp_df_sum_grid.describe())
print(shp_df_sum_grid.head())

# Save
shp_df_sum_grid.to_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_grid-sums.csv",index=False)

(10368, 7)
                LAT           LON   grid_df_id       area_km    n_articles  \
count  10368.000000  10368.000000  10368.00000  10368.000000  10368.000000   
mean       0.000000      0.000000   5183.50000  49196.033170     91.100887   
std       51.959018    103.925555   2993.12813  23773.819701    202.332822   
min      -88.750000   -178.750000      0.00000   1685.654015      0.000000   
25%      -44.375000    -89.375000   2591.75000  30341.895615      0.000000   
50%        0.000000      0.000000   5183.50000  54625.716973     10.000000   
75%       44.375000     89.375000   7775.25000  71049.402794     54.000000   
max       88.750000    178.750000  10367.00000  77252.429797   2637.000000   

       n_articles_weighted  
count         10368.000000  
mean              1.412519  
std               9.221334  
min               0.000000  
25%               0.000000  
50%               0.009322  
75%               0.220820  
max             295.271429  
     LAT     LON  grid_df

## Simple map to visualize

In [ ]:
## Plot of all articles using the weighted sum

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import colormaps
import matplotlib as mpl
    
# Read in the data    
shp_df_sum_grid = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_grid-sums.csv")
gridlons = shp_df_sum_grid.LON.unique()
gridlats = shp_df_sum_grid.LAT.unique()
shape = (len(gridlats), len(gridlons))
n = np.array(shp_df_sum_grid.n_articles_weighted).reshape(shape)

# quick check
plt.imshow(n, cmap='viridis')
plt.colorbar()
plt.show()


# set up a map
fig=plt.figure()
ax = plt.axes(projection=ccrs.Robinson())

# Can use the regular grid lons and lats with the shading ='nearest' option
im = plt.pcolormesh(gridlons, gridlats, n, shading = 'nearest',
              cmap=colormaps['magma_r'], norm=mpl.colors.LogNorm(), transform=ccrs.PlateCarree()) 

ax.coastlines()

## Colourbar
# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
cbar = plt.colorbar(im, cax=cax) 
cbar.ax.get_yaxis().labelpad = 15
cbar.set_label('Weighted # of articles', rotation=270)

# Save
plt.savefig(f'/home/dveytia/test-mordecai/figures/geoparsed-text-map.pdf',bbox_inches="tight") # Save plot, change file path and name
plt.show()


In [ ]:
## Plot of all articles in the ocean only using the weighted sum

# Read in the data    
shp_df_sum_grid = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_grid-sums.csv")

# Set everything on land to 0
shp_df_sum_grid_ocean = shp_df_sum_grid
shp_df_sum_grid_ocean.loc[shp_df_sum_grid_ocean['is_land'] == True, 'n_articles_weighted'] = 0

# Format into grid
gridlons = shp_df_sum_grid_ocean.LON.unique()
gridlats = shp_df_sum_grid_ocean.LAT.unique()
shape = (len(gridlats), len(gridlons))
n = np.array(shp_df_sum_grid_ocean.n_articles_weighted).reshape(shape)

# set up a map
fig=plt.figure()
ax = plt.axes(projection=ccrs.Robinson())

# Can use the regular grid lons and lats with the shading ='nearest' option
im = plt.pcolormesh(gridlons, gridlats, n, shading = 'nearest',
              cmap=colormaps['magma_r'], norm=mpl.colors.LogNorm(), transform=ccrs.PlateCarree()) 

ax.coastlines()

## Colourbar
# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
cbar = plt.colorbar(im, cax=cax) 
cbar.ax.get_yaxis().labelpad = 15
cbar.set_label('Weighted # of articles', rotation=270)

# Save
plt.savefig(f'/home/dveytia/test-mordecai/figures/geoparsed-text-ocean-map.pdf',bbox_inches="tight") # Save plot, change file path and name
plt.show()

In [ ]:
import pandas as pd
import os
import pandas as pd
import warnings
import numpy as np
import cartopy.crs as ccrs
import geopandas
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colormaps

## read in dataframe of grid matches
shp_df_sum_grid = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_grid-sums.csv")

## Plot

fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection=ccrs.Robinson())
ax.coastlines(lw=0.2)


shape = (len(shp_df_sum_grid.LAT.unique()), len(shp_df_sum_grid.LON.unique()))
n = np.array(shp_df_sum_grid.n_articles).reshape(shape)
mesh=ax.pcolormesh(
    shp_df_sum_grid.LON.unique(), 
    shp_df_sum_grid.LAT.unique(), 
    n, 
    cmap=colormaps['magma_r'],
    norm=mpl.colors.LogNorm(),
    transform=ccrs.PlateCarree(),
)

plt.show()

In [ ]:
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colormaps

## read in dataframe of grid matches
shp_df_sum_grid = pd.read_csv("/home/dveytia/test-mordecai/outputs/geoparsed-text_grid-sums.csv")
print(shp_df_sum_grid.shape)
print(shp_df_sum_grid.describe())

fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection=ccrs.Robinson())
ax.coastlines(lw=0.2)

x = shp_df_sum_grid['LON']
y = shp_df_sum_grid['LAT']
z = shp_df_sum_grid['n_articles']


c = plt.pcolormesh(x, y, z, cmap ='Greens')
plt.colorbar(c)
 
plt.title('matplotlib.pyplot.pcolormesh() function Example', fontweight ="bold")
plt.show()



(10368, 4)
                LAT           LON   grid_df_id    n_articles
count  10368.000000  10368.000000  10368.00000  10368.000000
mean       0.000000      0.000000   5183.50000     91.100887
std       51.959018    103.925555   2993.12813    202.332822
min      -88.750000   -178.750000      0.00000      0.000000
25%      -44.375000    -89.375000   2591.75000      0.000000
50%        0.000000      0.000000   5183.50000     10.000000
75%       44.375000     89.375000   7775.25000     54.000000
max       88.750000    178.750000  10367.00000   2637.000000


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
shape = (len(shp_df_sum_grid.LAT.unique()), len(shp_df_sum_grid.LON.unique()))
n = np.array(shp_df_sum_grid.n_articles).reshape(shape)
mesh=ax.pcolormesh(
    shp_df_sum_grid.LON.unique(), 
    shp_df_sum_grid.LAT.unique(), 
    n, 
    cmap=colormaps['magma_r'],
    norm=mpl.colors.LogNorm(),
    transform=ccrs.PlateCarree(),
)